In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [21]:
import re
import time
import pandas as pd
import numpy as np
import pymorphy2
from random import uniform
from tqdm.auto import tqdm
from bs4 import BeautifulSoup
from csv import DictWriter
from fake_useragent import UserAgent
from urllib.request import urlopen, Request
from urllib.error import HTTPError, URLError
from datetime import date
from dateutil.relativedelta import *
from datetime import datetime
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)

In [3]:
COLS_TO_CLEAN = ['LastPos', 'LastPos_2', 'LastPlace', 'LastPlace_2', 'LastUni', 'LastUni_2', 'Title', 'City']
COLS_TO_PREPROCESS = ['LastExpDesc', 'LastExpDesc_2', 'Description', 'Tags', 'Graduation', 'SpecCat']
TEXT_COLS = COLS_TO_CLEAN + COLS_TO_PREPROCESS

COLS_TO_FILL_ZERO = ['N_places', 'N_places_last_year', 'ExpPeriod_Months', 
                     'LastExp_Months', 'LastExp_2_Months', 'N_Unis', 'WorkExpCoef',
                     'hasHigherSchoolFinished', 'hasBachelorDeg', 'hasMastersDeg', 'hasDoctorDeg']
COLS_TO_FILL_MEAN = ['Age']
COLS_TO_FILL_MEDIAN = ['Salary']
COLS_TO_FILL_ONE = ['Gender']

COLS_TO_JOIN = ['LastExpDesc', 'LastExpDesc_2', 'Description', 'Tags']
COLS_TO_DROP = ['LastPos', 'LastPos_2', 'LastPlace', 'LastPlace_2', 'LastUni', 'LastUni_2', 'Title', 'City', 'SpecCat', 'URL']

DATA_ANALYST = 'DataAnalyst'
DATA_SCIENTIST = 'DataScientist'

In [4]:
# Loading dataset

df = pd.read_csv('data.csv', low_memory=False)
df

,DataAnalyst,DataScientist,Title,SpecCat,ExpPeriod_Months,Salary,Age,Gender,City,N_places,N_places_last_year,LastPlace,LastPlace_2,LastPos,LastPos_2,LastExp_Months,LastExp_2_Months,LastExpDesc,LastExpDesc_2,Description,Tags,Graduation,LastUni,LastUni_2,N_Unis,URL,FullTimeWork,Time2Work_Any,WorkExpCoef,HasSalary,Top12Uni,Top20Place,ExpPeriod_Years,LastExp_Years,LastExp_2_Years,N_extra_langs,Language_EN_Advanced,Language_RU_Native,WorkRights_Russia,WorkRights_Kazakhstan,WorkRights_Belarus,Citizenship_Russia,Citizenship_Kazakhstan,Citizenship_Belarus,WorkSchedule_FullDay,WorkSchedule_Remote,hasHigherSchoolFinished,hasBachelorDeg,hasMastersDeg,hasDoctorDeg
0,0.0,0.0,Консультант,"Продавец-консультант, продавец-кассир",117,NaN,34.0,1.0,Москва,11.0,1,"ООО ""НПО ""Спутник-А""","Ил, ОАО",Главный консультант по развитию проекта,Начальник отдела инициатив и инновационного ра...,49.0,9,Реализация федеральной программы по замене лиф...,Управление проектами на этапе инкубации:\r\n-Н...,Профессиональные навыки:\r\n\r\nПроектное упра...,"Оценка инвестиционных проектов, Оценка бизнеса...",Сертификат о завершении обучению торговле на в...,Российская академия народного хозяйства и госу...,"Российский университет дружбы народов, Москва",3.0,https://hh.ru/resume/1c76da3f00077f744f0039ed1...,1,1,0.094017,0,0,0,9.750000,4.083333,0.750000,3,1,1,1,0,0,1,0,0,0,0,1.0,0.0,0.0,0.0
1,0.0,0.0,"Head of Sales & Marketing , Sales Manager, Com...","Менеджер по продажам, менеджер по работе с кли...",385,NaN,NaN,0.0,Москва,4.0,1,Avia Reps,Air France-KLM,Senior Sales & Marketing Manager,Коммерческий представитель,48.0,296,Старший менеджер по продажам и маркетингу авиа...,Управление агентскими и корпоративными продажа...,Добрый день.\n\nЧтобы ознакомиться с краткой п...,"B2B Продажи, B2C продажи, Sales Planning, Sale...",NaN,Московский государственный лингвистический уни...,NaN,1.0,https://hh.ru/resume/75095b330006128b760039ed1...,1,1,0.010390,0,0,0,32.083333,4.000000,24.666667,3,1,1,1,0,0,1,0,0,1,0,1.0,0.0,0.0,0.0
2,0.0,0.0,Менеджер по работе с клиентами,"Менеджер по продажам, менеджер по работе с кли...",77,60000.0,36.0,0.0,Москва,7.0,0,"ООО ""Джанин Фитнес""","ФГБУ ""Дом отдыха ""Снегири"" УДП РФ",Менеджер по работе с клиентами,Менеджер,35.0,2,обеспечение работы по эффективному и культурно...,исследование рынка;\r\nопределение цен и ассор...,"могу работать с любым коллективом, быстро нахо...",NaN,NaN,Московский финансово-промышленный университет ...,NaN,1.0,https://hh.ru/resume/8b6cbc2b0002bb10a10039ed1...,1,1,0.090909,1,0,0,6.416667,2.916667,0.166667,1,0,1,1,0,0,1,0,0,0,0,1.0,0.0,0.0,0.0
3,0.0,0.0,Менеджер по обслуживанию юридических лиц,Кредитный специалист,141,50000.0,31.0,1.0,Москва,4.0,1,Оао кб еврокоммерц,Российский Союз Автостраховщиков,Менеджер по работе с клиентами,Экономист,127.0,5,"Обслуживание юридических лиц,открытие/закрытие...",Ответы на требования и запросы из государств о...,"Легкообучаемость,кропотливость, преданность к ...",Мед. книжка,NaN,ФА -- Финансовая академия при Правительстве РФ,КБГУ (Кабардино-Балкарский государственный уни...,2.0,https://hh.ru/resume/5cd430390004dcad4f0039ed1...,1,1,0.028369,1,0,0,11.750000,10.583333,0.416667,0,1,0,0,0,0,1,0,0,1,0,1.0,0.0,0.0,0.0
4,0.0,0.0,Экономист,Экономист,107,NaN,29.0,1.0,Москва,7.0,1,Consulting group,"ООО ""Леруа Мерлен"" Восток",Ведущий бухгалтер-экономист,Специалист по логистике,72.0,13,Работа с банками.\nРабота с налоговой.\nРабота...,Прием товаров на склад.\nАдресное размещение т...,"Ответственный, коммуникабельный, решительный, ...","Анализ инвестиционных проектов, Финансовое пла...",NaN,Московский финансово-промышленный университет ...,Киевский национальный экономический университе...,2.0,https://hh.ru/resume/f2e5b53e00032f40040039ed1...,1,1,0.065421,0,0,0,8.916667,6.000000,1.083333,2,1,1,1,0,0,1,0,0,1,0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [5]:
# Transforms text to lowercase
# Removes links
# Keeps only words of Russian and English characters
# Removes excess spaces

def clean_text(text):
  try:
    text = text.lower()
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^a-zа-яё]+', ' ', text)
    text = re.sub(r'ё+', 'е', text)
    text = re.sub(r"(?i)\b(?:оао|зао|ооо|ип|пао)\b", '', text)
    text = re.sub(r'\b\w{1}\b', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    if text == '':
      text = ' '
  except:
    text = ' '
  return text

In [6]:
# Deletes stopwords from NLTK library + my additional words

def delete_stopwords(text):
  try:
    text = text.split()
    stop_words = stopwords.words("english") + stopwords.words("russian")
    stop_words += ['добрый', 'день', 'hi', 'также', 'один', 'двое', 'трое', 
                   'четверо', 'пятеро', 'шестеро', 'семеро', 'вредный', 'привычка', 
                   'женатый', 'ребенок', 'женатый', 'школьный', 'возраст', 'другое']
    text = [word for word in text if not word in stop_words]
    text = ' '.join(text)
  except:
    text = ' '
  return text

In [7]:
# Converts each word to normal form

def normalize_words(text):
  try:
    text = text.split()
    morph = pymorphy2.MorphAnalyzer(lang='ru')
    text = [morph.parse(word)[0].normal_form for word in text]
    text = ' '.join(text)
  except:
    text = ' '
  return text

In [8]:
# Combines all text transformation into one function

def preprocess_text(text):
  text = clean_text(text)
  text = normalize_words(text)
  text = delete_stopwords(text)
  if text == '':
      text = ' '
  return text

In [9]:
for column in tqdm(COLS_TO_CLEAN, desc=f'Columns cleaned:'):
    df[column] = df[column].apply(clean_text)

for column in tqdm(COLS_TO_PREPROCESS, desc=f'Columns preprocessed:'):
    df[column] = df[column].apply(preprocess_text)

Columns cleaned::   0%|          | 0/8 [00:00<?, ?it/s]

Columns preprocessed::   0%|          | 0/6 [00:00<?, ?it/s]

In [117]:
try:
    df = pd.read_csv('preprocessed_data.csv', low_memory=False)
except:
    df.to_csv('preprocessed_data.csv', index=False)

In [118]:
df

,DataAnalyst,DataScientist,Title,SpecCat,ExpPeriod_Months,Salary,Age,Gender,City,N_places,N_places_last_year,LastPlace,LastPlace_2,LastPos,LastPos_2,LastExp_Months,LastExp_2_Months,LastExpDesc,LastExpDesc_2,Description,Tags,Graduation,LastUni,LastUni_2,N_Unis,URL,FullTimeWork,Time2Work_Any,WorkExpCoef,HasSalary,Top12Uni,Top20Place,ExpPeriod_Years,LastExp_Years,LastExp_2_Years,N_extra_langs,Language_EN_Advanced,Language_RU_Native,WorkRights_Russia,WorkRights_Kazakhstan,WorkRights_Belarus,Citizenship_Russia,Citizenship_Kazakhstan,Citizenship_Belarus,WorkSchedule_FullDay,WorkSchedule_Remote,hasHigherSchoolFinished,hasBachelorDeg,hasMastersDeg,hasDoctorDeg
0,0.0,0.0,консультант,продавец консультант продавец кассир,117,NaN,34.0,1.0,москва,11.0,1,нпо спутник,ил,главный консультант по развитию проекта,начальник отдела инициатив инновационного разв...,49.0,9,реализация федеральный программа замена лифтов...,управление проект этап инкубация основание ана...,профессиональный навык проектный управление ра...,оценка инвестиционный проект оценка бизнес оце...,сертификат завершение обучение торговля валютн...,российская академия народного хозяйства госуда...,российский университет дружбы народов москва,3.0,https://hh.ru/resume/1c76da3f00077f744f0039ed1...,1,1,0.094017,0,0,0,9.750000,4.083333,0.750000,3,1,1,1,0,0,1,0,0,0,0,1.0,0.0,0.0,0.0
1,0.0,0.0,head of sales marketing sales manager commerci...,менеджер продажа менеджер работа клиент,385,NaN,NaN,0.0,москва,4.0,1,avia reps,air france klm,senior sales marketing manager,коммерческий представитель,48.0,296,старший менеджер продажа маркетинг авиакомпани...,управление агентский корпоративный продажа укр...,ознакомиться краткий презентация пожалуйста пе...,продажа продажа sales planning sales skills sa...,,московский государственный лингвистический уни...,,1.0,https://hh.ru/resume/75095b330006128b760039ed1...,1,1,0.010390,0,0,0,32.083333,4.000000,24.666667,3,1,1,1,0,0,1,0,0,1,0,1.0,0.0,0.0,0.0
2,0.0,0.0,менеджер по работе клиентами,менеджер продажа менеджер работа клиент,77,60000.0,36.0,0.0,москва,7.0,0,джанин фитнес,фгбу дом отдыха снегири удп рф,менеджер по работе клиентами,менеджер,35.0,2,обеспечение работа эффективный культурный обсл...,исследование рынок определение цена ассортимен...,мочь работать любой коллектив быстро находить ...,,,московский финансово промышленный университет ...,,1.0,https://hh.ru/resume/8b6cbc2b0002bb10a10039ed1...,1,1,0.090909,1,0,0,6.416667,2.916667,0.166667,1,0,1,1,0,0,1,0,0,0,0,1.0,0.0,0.0,0.0
3,0.0,0.0,менеджер по обслуживанию юридических лиц,кредитный специалист,141,50000.0,31.0,1.0,москва,4.0,1,кб еврокоммерц,российский союз автостраховщиков,менеджер по работе клиентами,экономист,127.0,5,обслуживание юридический лицо открытие закрыти...,ответ требование запрос государство обслуживан...,легкообучаемость кропотливость преданность раб...,мёд книжка,,фа финансовая академия при правительстве рф,кбгу кабардино балкарский государственный унив...,2.0,https://hh.ru/resume/5cd430390004dcad4f0039ed1...,1,1,0.028369,1,0,0,11.750000,10.583333,0.416667,0,1,0,0,0,0,1,0,0,1,0,1.0,0.0,0.0,0.0
4,0.0,0.0,экономист,экономист,107,NaN,29.0,1.0,москва,7.0,1,consulting group,леруа мерлен восток,ведущий бухгалтер экономист,специалист по логистике,72.0,13,работа банк работа налоговый работа пенсионный...,приём товар склад адресный размещение товар за...,ответственный коммуникабельный решительный гот...,анализ инвестиционный проект финансовый планир...,,московский финансово промышленный университет ...,киевский национальный экономический университе...,2.0,https://hh.ru/resume/f2e5b53e00032f40040039ed1...,1,1,0.065421,0,0,0,8.916667,6.000000,1.083333,2,1,1,1,0,0,1,0,0,1,0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9724,1.0,1.0,technical project manager,руководитель проект аналитик учитель преподава...,112,NaN,28.0

In [119]:
for column in df.columns:
    if column in COLS_TO_FILL_ZERO:
        df[column] = df[column].fillna(0)
    elif column in COLS_TO_FILL_ONE:
        df[column] = df[column].fillna(1)
    elif column in COLS_TO_FILL_MEAN:
        df[column] = df[column].fillna(df[column].mean())
    elif column in COLS_TO_FILL_MEDIAN:
        df[column] = df[column].fillna(df[column].median())

In [120]:
df = df[df['Age'] < 80]
df = df[df['Salary'] < 1000000]
df = df.dropna(subset=[DATA_ANALYST, DATA_SCIENTIST])

In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9722 entries, 0 to 9728
Data columns (total 50 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   DataAnalyst              9722 non-null   float64
 1   DataScientist            9722 non-null   float64
 2   Title                    9722 non-null   object 
 3   SpecCat                  9722 non-null   object 
 4   ExpPeriod_Months         9722 non-null   int64  
 5   Salary                   9722 non-null   float64
 6   Age                      9722 non-null   float64
 7   Gender                   9722 non-null   float64
 8   City                     9722 non-null   object 
 9   N_places                 9722 non-null   float64
 10  N_places_last_year       9722 non-null   int64  
 11  LastPlace                9722 non-null   object 
 12  LastPlace_2              9722 non-null   object 
 13  LastPos                  9722 non-null   object 
 14  LastPos_2               

In [90]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9722 entries, 0 to 9728
Data columns (total 36 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   DataAnalyst              9722 non-null   float64
 1   DataScientist            9722 non-null   float64
 2   ExpPeriod_Months         9722 non-null   int64  
 3   Salary                   9722 non-null   float64
 4   Age                      9722 non-null   float64
 5   Gender                   9722 non-null   float64
 6   N_places                 9722 non-null   float64
 7   N_places_last_year       9722 non-null   int64  
 8   LastExp_Months           9722 non-null   float64
 9   LastExp_2_Months         9722 non-null   int64  
 10  N_Unis                   9722 non-null   float64
 11  FullTimeWork             9722 non-null   int64  
 12  Time2Work_Any            9722 non-null   int64  
 13  WorkExpCoef              9722 non-null   float64
 14  HasSalary               

In [91]:
df_copy = df[[DATA_ANALYST, DATA_SCIENTIST]]
NUMERIC_COLS = df.select_dtypes(include='number').columns.tolist()[2:]
df_copy[NUMERIC_COLS] = df[NUMERIC_COLS]
df_copy['Text'] = df[COLS_TO_JOIN].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)


/var/folders/05/g_42ptlx3vqgr85jtt5kpthw0000gn/T/ipykernel_87696/3978817007.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy[NUMERIC_COLS] = df[NUMERIC_COLS]
/var/folders/05/g_42ptlx3vqgr85jtt5kpthw0000gn/T/ipykernel_87696/3978817007.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy[NUMERIC_COLS] = df[NUMERIC_COLS]
/var/folders/05/g_42ptlx3vqgr85jtt5kpthw0000gn/T/ipykernel_87696/3978817007.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


In [93]:
y_analyst = df_copy[DATA_ANALYST]
y_scientist = df_copy[DATA_SCIENTIST]
X = df_copy.drop(columns=[DATA_ANALYST, DATA_SCIENTIST])

In [140]:
X_num = X[NUMERIC_COLS].to_numpy()

numpy.ndarray

In [183]:
X_vec = TfidfVectorizer().fit_transform(X['Text'].to_numpy()).toarray()

In [184]:
X_all = np.concatenate((X_num, X_vec), axis=1)

array([1.17000000e+02, 8.40000000e+04, 3.40000000e+01, 1.00000000e+00,
       1.10000000e+01, 1.00000000e+00, 4.90000000e+01, 9.00000000e+00,
       3.00000000e+00, 1.00000000e+00, 1.00000000e+00, 9.40170940e-02,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 9.75000000e+00,
       4.08333333e+00, 7.50000000e-01, 3.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [173]:
X_train, X_test, y_train, y_test = train_test_split(X_all, y_analyst.to_numpy(), test_size=0.2, random_state=42)

In [170]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV, StratifiedKFold

In [174]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', RandomForestClassifier())
])

# Grid Search Parameters for RandomForest
param_grid = {'clf__n_estimators': np.linspace(1, 100, 10, dtype=int),
              'clf__min_samples_split': [3, 10],
              'clf__min_samples_leaf': [3],
              'clf__max_features': [7],
              'clf__max_depth': [None],
              'clf__criterion': ['gini'],
              'clf__bootstrap': [False]}

# Training config
kfold = StratifiedKFold(n_splits=3)
scoring = {'Accuracy': 'accuracy', 'F1': 'f1_macro'}
refit = 'F1'

# Perform GridSearch
rf_model = GridSearchCV(pipeline, param_grid=param_grid, cv=kfold, scoring=scoring, 
                         refit=refit, n_jobs=-1, return_train_score=True, verbose=1)
rf_model.fit(X_train, y_train)
rf_best = rf_model.best_estimator_

Fitting 7 folds for each of 20 candidates, totalling 140 fits


KeyboardInterrupt: 

In [21]:
# try:
#     analyst_df = pd.read_csv('dataset_analyst.csv', low_memory=False)
#     scientist_df = pd.read_csv('dataset_scientist.csv', low_memory=False)
# except:
    # analyst_df.to_csv('dataset_analyst.csv', index=False)
    # scientist_df.to_csv('dataset_scientist.csv', index=False)

In [22]:
# NUMERIC_COLS = df.select_dtypes(include='number').columns.tolist()[2:]
# TEXT_COLS = COLS_TO_CLEAN + COLS_TO_PREPROCESS

In [ ]:
df_copy

In [129]:
X_text = df[TEXT_COLS]
X_numeric = df[NUMERIC_COLS].values
X = pd.concat([X_text, pd.DataFrame(X_numeric, columns=NUMERIC_COLS)], axis=1)
y = y_DataAnalyst


In [130]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Make predictions on the test set
y_pred = pipeline.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)